In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
import seaborn as sns
from IPython.display import Image
import os
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from math import sqrt
from sklearn import linear_model
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import pipeline, preprocessing, neighbors, model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


/Users/justinblinder/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load cleaned dataset
df_pluto = pd.read_pickle('../data/pkl/df_pluto-cat-feat-2013-2017.pkl')

# Create holdout set of current vacant lots that are in specific zip code
#df_pluto_holdout = df_pluto[(df_pluto['LandUse']==11)&(df_pluto['PolicePrct_75.0'] == 1)]

# Remove columns from train/test set
df_pluto = df_pluto[df_pluto.columns.difference(['BldgClass','FireComp','IrrLotCode','Sanborn','SplitZone','ZoneDist1'])]
df_pluto = df_pluto[df_pluto.columns.difference(['APPBBL','AreaSource','BBL','BoroCode','CondoNo','TaxMap'])]
df_pluto = df_pluto[df_pluto.columns.difference(['new_bldg_prob','ZoneCodeChanged','LandUse','LandUse2016','ZoneCodeBecameCondo'])]
df_pluto = df_pluto[df_pluto.columns.difference(['Cat_LandUse2016_11.0','Cat_LandUse2016_10.0','Cat_LandUse2016_9.0','Cat_LandUse2016_8.0','Cat_LandUse2016_7.0','Cat_LandUse2016_6.0','Cat_LandUse2016_5.0','Cat_LandUse2016_4.0','Cat_LandUse2016_3.0','Cat_LandUse2016_2.0','Cat_LandUse2016_1.0'])]
df_pluto = df_pluto.dropna()

In [3]:
from imblearn.over_sampling import SMOTE 

# Subset data
df_pluto_subset = df_pluto[::50]

# Get coefs
y = df_pluto_subset['ZoneCodeWasVacant'].astype(int)
X = df_pluto_subset[df_pluto_subset.columns.difference(['ZoneCodeWasVacant'])]

# Create test train split
xtrain, xtest, ytrain, ytest= train_test_split(X,y)

# Smote train/ test data
sm = SMOTE(random_state=42)
xtrain, ytrain = sm.fit_sample(xtrain,ytrain)

In [4]:
# Scale train/test/holdout values values
scaler = StandardScaler().fit(xtrain)
xtrain_s = scaler.transform(xtrain)
xtest_s = pd.DataFrame(scaler.transform(xtest))


In [5]:
import pickle
with open(r"../data/pkl/rf-alldata-final-scaled.pkl", "rb") as input_file:
    rf = pickle.load(input_file)

In [ ]:
# Grid Search
from sklearn.model_selection import GridSearchCV

#build parameter grid
n_estimators_range = list(range(10,100,10))
max_features_options = ['sqrt', 'log2', None]
min_samples_split_range = list(np.linspace(0.01,.1,5))
min_impurity_decrease_range = list(np.linspace(0.01,.1,5))
tuned_parameters = {'C': [0.1, 0.5, 1, 5, 10, 50, 100]}

param_grid = dict(n_estimators = n_estimators_range,
                  max_features = max_features_options,
                  min_samples_split = min_samples_split_range, 
                  min_impurity_decrease = min_impurity_decrease_range)

#fit grid
grid = GridSearchCV(RandomForestClassifier(n_jobs=-1), param_grid, scoring='accuracy', n_jobs=-1)
grid.fit(xtrain_s,ytrain)

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)